# PDH Feedback Control

This notebook is essentially a paraphrasing of [this document](../Resources/pdh_control.pdf) along with some additional justifications to explain why some things are the way they are. The paper should be used as the validation mechanism if anything written here seems off.


## 1.1: Basic Concepts

Define the following terms:

$$
\hat{A}(s) = \mathcal{L}\{h(t)\} = \frac{Y(s)}{X(s)}
$$

Where $s = \sigma + 2\pi f = \sigma + j\omega$. Because we are analyzing the closed-loop behaviour of the system we set $\sigma$ equal to 0. We can do this because it's a closed loop, so things like an unstable impulse response where a positive $\sigma$ would cause the system output to become unstable is already encoded in the closed-loop transfer function evaluated along the imaginary axis. In other words, we can work purely in the frequency domain when evaluating the closed-loop behaviour of this system.

The gain and phase of $\hat{A}$ are defined as follows:

$$
|\hat{A}|, \angle{\hat{A}}
$$


Loop components perform algebraic operations on their inputs in the frequency domain. These operations correspond to convolutions in the time domain. The open-loop transfer function is the product of all loop components:

$$
\alpha = \hat{K}\hat{G}\hat{H}
$$

In the time domain this represents the end-to-end convolution applied to the input signal. We use a negative feedback loop because we need a way to self-regulate our current response as a function of our priors. Assuming the last output is fed as negative feedback to be subtracted from our input signal before passing through the chain, we can define the closed-loop transfer function as follows:

$$
e = x - y\newline
y = \alpha e = \alpha(x - y)\newline
(1 + \alpha)y = \alpha x\newline
\frac{y}{x} = \frac{\alpha}{1 + \alpha}
$$

This is super unintuitive unless you treat $y_{o, n}$ and $y_{e, n-1}$ as the same term $y$. We can do this because our system is assumed to be linear time-invariant (LTI). We make this assumption because our transfer function does not change with respect to time. By definition of what the laplace transform does, no frequency domain signals will change with respect to time. This means niether X(s), Y(s), nor H(s) will change with respect to whatever cycle of the loop we are on. It also makes the concept of loop cycles irrelevant because doing this in the frequency domain is inherently equivalent to integrating this recursive feedback loop across all of time. This means that our feedback loop modelled in the laplace domain really just models the convergence of a series in the time domain where each term in that series is a convolution between our transfer function and our input.

Let $h(t)$ represent the open-loop impulse response such that a single pass $y_{\text{open}}(t) = (h * x)(t)$


$$
y_0(t) = 0\newline
e_1(t) = x(t) - y_0(t) = x(t)\newline
y_1(t) = (h * e_1)(t) = (h * x)(t)\newline
e_2(t) = x(t) - y_1(t)\newline
y_2(t) = (h * e_2)(t) = (h * x)(t) - (h * h * x)(t)\newline
e_3(t) = x(t) - y_2(t)\newline
y_3(t) = (h * x)(t) - (h * h * x)(t) + (h * h * h * x)(t)\newline
$$


Through induction we get the following series:

$$
y_n(t) = \sum_{k=1}^n (-1)^{k-1}(h^{*k} * x)(t)
$$

The steady-state solution is the limit:

$$
y(t) = \lim_{n\rightarrow\infty}y_n(t) = \sum_{k=1}^\infty (-1)^{k-1}(h^{*k} * x)(t)\newline
$$

We can also map this series to its Laplace domain counterpart:

$$
\mathcal{L}\{h^{*k}(t)\} = [H(s)]^k\newline
Y(s) = \sum_{k=1}^\infty(-1)^{k-1}H(s)^kX(s) = \frac{H(s)}{1 + H(s)}X(s)
$$

The Laplace domain representation gives us a geometric series that converges for all $|H(s)| < 1$. In other words, it will converge as long as the open-loop transfer function does not boost the signal. The frequency domain representation is the sum of the infinite series of convolutions that represent the limiting behaviour of our feedback process. 

Real-world systems have a delay between the input and the output. We can introduce $\tau$-shifted delta convolutions into each step of our series to account for this:

$$
h_\tau = h(t) * \delta(t-\tau)
y_n(t) = \sum_{k=1}^n(-1)^{k-1}(h * h_\tau^{*(k-1)} * x)(t)\newline

$$

Via $\mathcal{L}\{\delta(t - \tau)\} = e^{-s\tau}$

$$
Y(s) = H(s)X(s)\sum_{k=0}^{\infty}(-1)^kH^k(s)e^{-s\tau k}
$$

If $|H(s)e^{-s\tau}| < 1$, this geometric series converges to
 
$$
Y(s) = \frac{H(s)}{1 + H(s)e^{-s\tau}}X(s)
$$

The stability of this closed-loop expression is determined by the roots of $1 + H(s)e^{-s\tau} = 0$.


This was a bit off-topic and rather tedious, but I think it's much more powerful and rewarding to think of control theory as a form of applied recursion rather than just memorizing a bunch of block diagram hacks to be implemented through black-box Matlab code.